# Amazon S3 Integration

## Read JSON from S3

In [0]:
# Set AWS Credentials in Spark
aws_access_key = dbutils.secrets.get(scope="aws-credentials", key="aws-access-key")
aws_secret_key = dbutils.secrets.get(scope="aws-credentials", key="aws-secret-key")
spark.conf.set("fs.s3a.access.key", aws_access_key)
spark.conf.set("fs.s3a.secret.key", aws_secret_key)
spark.conf.set("fs.s3a.endpoint", "s3.ap-southeast-2.amazonaws.com")  # Sydney region

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-746201911757538>, line 4
      2 aws_access_key = dbutils.secrets.get(scope="aws-credentials", key="aws-access-key")
      3 aws_secret_key = dbutils.secrets.get(scope="aws-credentials", key="aws-secret-key")
----> 4 spark.conf.set("fs.s3a.access.key", aws_access_key)
      5 spark.conf.set("fs.s3a.secret.key", aws_secret_key)
      6 spark.conf.set("fs.s3a.endpoint", "s3.ap-southeast-2.amazonaws.com")

File /databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/conf.py:46, in RuntimeConf.set(self, key, value)
     44 op_set = proto.ConfigRequest.Set(pairs=[proto.KeyValue(key=key, value=value)])
     45 operation = proto.ConfigRequest.Operation(set=op_set)
---> 46 result = self._client.config(operation)
     47 for warn in result.warnings:
     48     warnings.warn(warn)

File /databricks/python/lib/python3.10

In [0]:
# Mount S3 bucket to DBFS

aws_access_key = dbutils.secrets.get(scope="aws-credentials", key="aws-access-key")
aws_secret_key = dbutils.secrets.get(scope="aws-credentials", key="aws-secret-key")
bucket_name = "ernest-aws-bucket"
mount_point = "/mnt/stock-data"
try:
    dbutils.fs.mount(
        source=f"s3a://{bucket_name}",
        mount_point=mount_point,
        extra_configs={
            "fs.s3a.access.key": aws_access_key,
            "fs.s3a.secret.key": aws_secret_key,
            "fs.s3a.endpoint": "s3.ap-southeast-2.amazonaws.com"
        }
    )
    print(f"Mounted {bucket_name} to {mount_point}")
except Exception as e:
    print(f"Mount failed: {e}")

Mount failed: An error occurred while calling o933.mount. Trace:
py4j.security.Py4JSecurityException: Method public com.databricks.backend.daemon.dbutils.DBUtilsCore$Result com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(java.lang.String,java.lang.String,java.lang.String,java.lang.String,java.util.Map) is not whitelisted on class class com.databricks.backend.daemon.dbutils.DBUtilsCore
	at py4j.security.WhitelistingPy4JSecurityManager.checkCall(WhitelistingPy4JSecurityManager.java:473)
	at py4j.Gateway.invoke(Gateway.java:305)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:199)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:119)
	at java.lang.Thread.run(Thread.java:750)




In [0]:
# Access S3 via Catalog
from pyspark.sql import SparkSession

# Initialise Spark session
spark = SparkSession.builder.appName("StockDataTransform").getOrCreate()

# Define S3 path via external location
bucket_name = "ernest-aws-bucket"
raw_s3_path = f"s3://{bucket_name}/raw/data/*.json"

# Read JSON files
try:
    raw_df = spark.read.json(raw_s3_path)
except Exception as e:
    print(f"Error reading from S3: {e}")

In [0]:
raw_df.printSchema()

root
 |-- Meta Data: struct (nullable = true)
 |    |-- 1. Information: string (nullable = true)
 |    |-- 2. Symbol: string (nullable = true)
 |    |-- 3. Last Refreshed: string (nullable = true)
 |    |-- 4. Output Size: string (nullable = true)
 |    |-- 5. Time Zone: string (nullable = true)
 |-- Time Series (Daily): struct (nullable = true)
 |    |-- 2024-10-18: struct (nullable = true)
 |    |    |-- 1. open: string (nullable = true)
 |    |    |-- 2. high: string (nullable = true)
 |    |    |-- 3. low: string (nullable = true)
 |    |    |-- 4. close: string (nullable = true)
 |    |    |-- 5. volume: string (nullable = true)
 |    |-- 2024-10-21: struct (nullable = true)
 |    |    |-- 1. open: string (nullable = true)
 |    |    |-- 2. high: string (nullable = true)
 |    |    |-- 3. low: string (nullable = true)
 |    |    |-- 4. close: string (nullable = true)
 |    |    |-- 5. volume: string (nullable = true)
 |    |-- 2024-10-22: struct (nullable = true)
 |    |    |-- 1.

## Transform the data

In [0]:
from pyspark.sql.functions import col, lit

# Create a DataFrame with date and values as separate columns
time_series_df = raw_df.select("Time Series (Daily).*")
date_columns = time_series_df.columns
rows = []
for date in date_columns:
    rows.append(
        time_series_df.select(
            lit(date).alias("date"),
            col(date).alias("values")
        )
    )

# Union all rows into one DataFrame
flattened_df = rows[0]
for row_df in rows[1:]:
    flattened_df = flattened_df.union(row_df)

flattened_df.show(5, truncate=False)

+----------+--------------------------------------------------+
|date      |values                                            |
+----------+--------------------------------------------------+
|2024-10-18|{236.1800, 236.1800, 234.0100, 235.0000, 46431472}|
|2024-10-21|{234.4500, 236.8500, 234.4500, 236.4800, 36254470}|
|2024-10-22|{233.8850, 236.2200, 232.6000, 235.8600, 38846578}|
|2024-10-23|{234.0800, 235.1440, 227.7600, 230.7600, 52286979}|
|2024-10-24|{229.9800, 230.8200, 228.4100, 230.5700, 31109503}|
+----------+--------------------------------------------------+
only showing top 5 rows


In [0]:
clean_df = flattened_df.select(
    col("date"),
    col("values.`1. open`").cast("float").alias("open"),
    col("values.`2. high`").cast("float").alias("high"),
    col("values.`3. low`").cast("float").alias("low"),
    col("values.`4. close`").cast("float").alias("close"),
    col("values.`5. volume`").cast("long").alias("volume")
)

# Add a simple transformation
clean_df = clean_df.withColumn("daily_range", col("high") - col("low"))

clean_df.show(5, truncate=False)

+----------+-------+-------+------+------+--------+-----------+
|date      |open   |high   |low   |close |volume  |daily_range|
+----------+-------+-------+------+------+--------+-----------+
|2024-10-18|236.18 |236.18 |234.01|235.0 |46431472|2.1699982  |
|2024-10-21|234.45 |236.85 |234.45|236.48|36254470|2.4000092  |
|2024-10-22|233.885|236.22 |232.6 |235.86|38846578|3.619995   |
|2024-10-23|234.08 |235.144|227.76|230.76|52286979|7.3840027  |
|2024-10-24|229.98 |230.82 |228.41|230.57|31109503|2.4100037  |
+----------+-------+-------+------+------+--------+-----------+
only showing top 5 rows


## Upload back to S3

In [0]:
file_key = "processed/data/AAPL.parquet"
output_s3_path = f"s3://{bucket_name}/{file_key}"
clean_df.write.mode("overwrite").parquet(output_s3_path)

verify_df = spark.read.parquet(output_s3_path)
verify_df.show(5, truncate=False)

+----------+-------+-------+------+------+--------+-----------+
|date      |open   |high   |low   |close |volume  |daily_range|
+----------+-------+-------+------+------+--------+-----------+
|2024-10-18|236.18 |236.18 |234.01|235.0 |46431472|2.1699982  |
|2024-10-21|234.45 |236.85 |234.45|236.48|36254470|2.4000092  |
|2024-10-22|233.885|236.22 |232.6 |235.86|38846578|3.619995   |
|2024-10-23|234.08 |235.144|227.76|230.76|52286979|7.3840027  |
|2024-10-24|229.98 |230.82 |228.41|230.57|31109503|2.4100037  |
+----------+-------+-------+------+------+--------+-----------+
only showing top 5 rows
